In [ ]:
import cv2
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
from moviepy.editor import AudioFileClip, concatenate_videoclips, TextClip, CompositeVideoClip, VideoFileClip, ImageClip
import os

# Use relative path that works across environments
audio_file = 'bark_generation.wav'
if not os.path.exists(audio_file):
    audio_file = 'bark_generation_female.wav'

if not os.path.exists(audio_file):
    print(f"Error: Audio file not found. Please run bark_ai.ipynb first to generate audio files.")
    exit(1)

try:
    # Load audio file
    y, sr = librosa.load(audio_file, sr=44100)

    # Compute mel spectrogram
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128, fmax=8000)
    S_dB = librosa.power_to_db(S, ref=np.max)

    # Generate spectrogram visualization
    fig, ax = plt.subplots(figsize=(12, 4))
    img = librosa.display.specshow(S_dB, x_axis='time', y_axis='mel', sr=sr, ax=ax)
    fig.canvas.draw()

    # Extract the image and create a VideoClip object from it
    spectrogram = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    spectrogram = spectrogram.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    spectrogram_clip = ImageClip(spectrogram, duration=librosa.get_duration(y))

    # Generate waveform visualization
    fig, ax = plt.subplots(figsize=(12, 4))
    librosa.display.waveform(y, sr=sr, ax=ax)
    fig.canvas.draw()

    # Extract the image and create a VideoClip object from it
    waveform = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    waveform = waveform.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    waveform_clip = ImageClip(waveform, duration=librosa.get_duration(y))

    # Create a simple colored background instead of requiring silent.mp4
    from moviepy.editor import ColorClip
    background_clip = ColorClip(size=(640, 480), color=(0, 0, 0), duration=librosa.get_duration(y))

    # Combine the background with the spectrogram and waveform clips
    final_clip = CompositeVideoClip([background_clip, spectrogram_clip.set_position('center')])
    final_clip = final_clip.set_audio(AudioFileClip(audio_file))

    # Write the final clip to a video file
    final_clip.write_videofile('podcast_visualizer.mp4', audio_codec='aac')
    print("Video generated successfully: podcast_visualizer.mp4")

except Exception as e:
    print(f"Error generating video: {str(e)}")
    print("Make sure all required dependencies are installed and audio files exist.")

In [6]:
!pip install moviepy librosa numpy matplotlib
!pip install opencv-python